In [9]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import matplotlib.pyplot as plt

In [10]:
# 载入数据
mnist = input_data.read_data_sets('mnist_data',one_hot=True)
training = mnist.train.images
trainlabel = mnist.train.labels
testing = mnist.test.images
testlabel = mnist.test.labels

Extracting mnist_data\train-images-idx3-ubyte.gz
Extracting mnist_data\train-labels-idx1-ubyte.gz
Extracting mnist_data\t10k-images-idx3-ubyte.gz
Extracting mnist_data\t10k-labels-idx1-ubyte.gz


In [11]:
# 定义网络结构 784 * 256 * 128 * 10
n_input = 784
n_hidden_1 = 256
n_hidden_2 = 128
n_classes = 10

# 输入变量
x = tf.placeholder(tf.float32,[None,n_input])
y = tf.placeholder(tf.float32,[None,n_classes])

#
stddev = 0.1
weights = {
    'w1' : tf.Variable(tf.random_normal([n_input,n_hidden_1],stddev=stddev)),
    'w2' : tf.Variable(tf.random_normal([n_hidden_1,n_hidden_2],stddev=stddev)),
    'out' : tf.Variable(tf.random_normal([n_hidden_2,n_classes],stddev=stddev))
}
biases = {
    'b1' : tf.Variable(tf.random_normal([n_hidden_1])),
    'b2' : tf.Variable(tf.random_normal([n_hidden_2])),
    'out' : tf.Variable(tf.random_normal([n_classes]))
}

In [12]:
# 定义网络结构
def multilayer_perceptron(_X,_weights,_biases):
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(_X,_weights['w1']),_biases['b1']))
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1,_weights['w2']),_biases['b2']))
    layer_out = tf.matmul(layer_2,_weights['out'])
    return layer_out

In [13]:
# 预测结果
pred = multilayer_perceptron(x,weights,biases)

# 损失函数，优化器
with tf.device('/gpu:0'):
    # 损失函数：交叉熵函数
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred,labels=y))
    optm = tf.train.AdamOptimizer().minimize(cost)
    corr = tf.equal(tf.argmax(pred,1),tf.argmax(y,1))
    accr = tf.reduce_mean(tf.cast(corr,tf.float32)) 


In [17]:
# 循环参数
training_epochs = 20
batch_size = 100
display_step = 4

config = tf.ConfigProto()  
config.gpu_options.per_process_gpu_memory_fraction = 0.5  # 程序最多只能占用指定gpu50%的显存  
config.gpu_options.allow_growth = True      #程序按需申请内存  
sess = tf.Session(config = config) 
# 迭代
with tf.Session(config=config ) as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(training_epochs):
        avg_cost = 0.0
        total_batch = (mnist.train.num_examples // batch_size)
        # 批次循环
        for i in range(total_batch):
            batch_x,batch_y = mnist.train.next_batch(batch_size=batch_size)
            feeds = {x:batch_x,y:batch_y}
            # 进行一次梯度下降训练
            sess.run(optm,feed_dict=feeds)
            # 计算本次的代价，并累计
            avg_cost += sess.run(cost,feed_dict=feeds)
        avg_cost = avg_cost / total_batch
        # 显示中间结果
        if (epoch + 1) % display_step == 0:
            print('Epoch: %03d / %03d cost: %0.9f' 
                  % ( epoch,training_epochs,avg_cost ))
            # 计算训练集准确度，和测试集准确度
            feeds = {x:batch_x,y:batch_y} 
            train_acc = sess.run(accr,feed_dict=feeds)
            print('\\t训练集准确度：%.3f' ,train_acc)
            feeds = {x:mnist.test.images,y:mnist.test.labels} 
            test_acc = sess.run(accr,feed_dict=feeds)
            print('\\t测试集准确度：%.3f' ,test_acc)
    pass

print('Close')


Epoch: 003 / 020 cost: 0.118128967
\t训练集准确度：%.3f 0.98
\t测试集准确度：%.3f 0.9622
Epoch: 007 / 020 cost: 0.046130258
\t训练集准确度：%.3f 1.0
\t测试集准确度：%.3f 0.9746
Epoch: 011 / 020 cost: 0.018121993
\t训练集准确度：%.3f 1.0
\t测试集准确度：%.3f 0.9797
Epoch: 015 / 020 cost: 0.006954740
\t训练集准确度：%.3f 1.0
\t测试集准确度：%.3f 0.9802
Epoch: 019 / 020 cost: 0.002611529
\t训练集准确度：%.3f 1.0
\t测试集准确度：%.3f 0.9758
Close
